#### -----------------------------------------------------------------------------<br>Copyright (c) 2024, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [1]:
import time
from arena_api.system import system

#### Acquisition: Introduction
>    This example introduces the basics of image acquisition. This includes
    setting image acquisition and buffer handling modes, setting the device to
    automatically negotiate packet size, and setting the stream packet resend
    node before starting the image stream. The example then starts acquiring
    images by grabbing and requeuing buffers, and retrieving data on images
    before finally stopping the image stream.

In [2]:
"""
This function waits for the user to connect a device before raising an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
            tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

device = system.select_device(devices)
nodemap = device.nodemap
tl_stream_nodemap = device.tl_stream_nodemap
print(f'Device used in the example:\n\t{device}')

Created 1 device(s)
  Only one device detected:  ('1c:0f:af:4c:c3:c7', 'HTW003S-001', '', '169.254.200.195')
    Automatically selecting this device.
Device used in the example:
	('1c:0f:af:4c:c3:c7', 'HTW003S-001', '', '169.254.200.195')


##### Store initial values
> These initial values are restored to the device after the example is completed

In [3]:
print(f"Store initial values")
initial_acquisition_mode = nodemap.get_node("AcquisitionMode").value
width_initial = nodemap.get_node("Width").value
height_initial = nodemap.get_node("Height").value

Store initial values


##### Configure device settings

In [4]:
nodemap.get_node("AcquisitionMode").value = "Continuous"

##### Set buffer handling mode
>  Set buffer handling mode before starting the stream. Starting the
    stream requires the buffer handling mode to be set beforehand. The
    buffer handling mode determines the order and behavior of buffers in
    the underlying stream engine. Setting the buffer handling mode to
    'NewestOnly' ensures the most recent image is delivered, even if it
    means skipping frames.

In [5]:
tl_stream_nodemap["StreamBufferHandlingMode"].value = "NewestOnly"

##### Enable stream auto negotiate packet size
> Setting the stream packet size is done before starting the stream.
    Setting the stream to automatically negotiate packet size instructs the
    camera to receive the largest packet size that the system will allow.
    This generally increases frame rate and results in fewer interrupts per
    image, thereby reducing CPU load on the host system. Ethernet settings
    may also be manually changed to allow for a larger packet size.

In [6]:
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

##### Enable stream packet resend
> Enable stream packet resend before starting the stream. Images are sent
    from the camera to the host in packets using UDP protocol, which
    includes a header image number, packet number, and timestamp
    information. If a packet is missed while receiving an image, a packet
    resend is requested and this information is used to retrieve and
    redeliver the missing packet in the correct order.

In [7]:
tl_stream_nodemap['StreamPacketResendEnable'].value = True

##### Set features before streaming.-------------------------------------------
>  Set width and height to their max values

In [8]:
# print("Getting 'Width' and 'Height' Nodes")
# width_node = device.nodemap.get_node('Width')
# height_node = device.nodemap.get_node('Height')

# if width_node is None or height_node is None:
#     raise Exception('Width or Height node not found in the nodemap.')

# print("Setting 'Width' and 'Height' Nodes value to their max values")
# width_node.value = width_node.max
# height_node.value = height_node.max

##### Grab images -------------------------------------------------------------
> - Starting the stream allocates buffers, which can be passed in as
 an argument (default: 10), and begins filling them with data.
 Buffers must later be requeued to avoid memory leaks.<br>
> - 'Device.get_buffer()' with no arguments returns one buffer(NOT IN A LIST)<br>
> - 'Device.get_buffer(30)' returns 30 buffers(IN A LIST)<br>
> - 'Device.requeue_buffer()' takes a buffer or many buffers in a list or tuple

In [9]:
import os
os.chdir('D:\LiDAR Capture')
from arena_api.__future__.save import Writer  # Import the Writer class
from arena_api.system import system

number_of_buffers = 1000

# Assume device initialization and selection here
devices = system.create_device()
device = system.select_device(devices)

device.start_stream(number_of_buffers)
print(f'Stream started with {number_of_buffers} buffers')

print(f'Get {number_of_buffers} buffers in a list')
buffers = []
for _ in range(number_of_buffers):
    buffer = device.get_buffer()
    buffers.append(buffer)
print("Success")

# Create a Writer instance
writer = Writer()

# Create a directory to save PLY files if it doesn't exist
output_folder = '3D_data_output'
os.makedirs(output_folder, exist_ok=True)

'''
Print image buffer info
    Buffers contain image data.
    Image data can also be copied and converted using BufferFactory.
    That is necessary to retain image data, as we must also requeue the buffer.
'''
for count, buffer in enumerate(buffers):
    print(f'\tbuffer{count:{2}} received | '
          f'Width = {buffer.width} pxl, '
          f'Height = {buffer.height} pxl, '
          f'Pixel Format = {buffer.pixel_format.name}')
    
    filename = os.path.join(output_folder, f'data_{count}.ply')
    writer.save(buffer, filename)
    device.requeue_buffer(buffer)
    print(f'\tbuffer{count:{2}} requeued')

print(f'Requeued {number_of_buffers} buffers')

device.stop_stream()
print('Stream stopped')




  Only one device detected:  ('1c:0f:af:4c:c3:c7', 'HTW003S-001', '', '169.254.200.195')
    Automatically selecting this device.
Stream started with 1000 buffers
Get 1000 buffers in a list
Success
	buffer 0 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 0 requeued
	buffer 1 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 1 requeued
	buffer 2 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 2 requeued
	buffer 3 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 3 requeued
	buffer 4 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 4 requeued
	buffer 5 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 5 requeued
	buffer 6 received | Width = 640 pxl, Height = 480 pxl, Pixel Format = Coord3D_ABCY16
	buffer 6 requeued
	buffer 7 received | Width = 640 pxl, Height = 480 pxl, Pixel Forma

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [10]:
# nodemap.get_node("AcquisitionMode").value = initial_acquisition_mode
# nodemap.get_node("Width").value = width_initial
# nodemap.get_node("Height").value = height_initial

system.destroy_device()
print('Destroyed all created devices')

Destroyed all created devices
